In [35]:
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Input
from tensorflow.keras.models import Model, Sequential, save_model, load_model
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.utils import shuffle

from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam, SGD

from keras.utils import np_utils

import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import os
import glob

import scikitplot as skplt
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

import pickle

In [28]:
fh = open('/Users/home/Documents/Respiratory Illness AI/randomness_test/recording/test0/asthma_others0_224_224_3', 'rb')
dataset = pickle.load(fh)

In [29]:
def create_model(input_shape):
    baseModel = VGG19(weights=None, include_top=False, input_shape=input_shape)

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
    headModel = Flatten(name="flatten")(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(512, activation='relu')(headModel)
    headModel = Dropout(0.5)(headModel)

    headModel = Dense(2, activation="sigmoid")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False
        
    return model 

In [36]:
input_shape = (224, 224, 3)

n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True)
X, Y = zip(*dataset)

X_data, Y_data = np.array(X), np.array(Y)
X_data, Y_data = shuffle(X_data, Y_data)

In [37]:
splits = [[train, test] for train, test in kfold.split(X_data,Y_data)]

train = splits[0][0]
test = splits[0][1]

In [38]:
x_train, x_val = X_data[train], X_data[test]
y_train, y_val = Y_data[train], Y_data[test]

x_train = np.array([x.reshape(input_shape) for x in x_train])
x_val = np.array([x.reshape(input_shape) for x in x_val])
#remember to change number of classes 
y_train = np.array(np_utils.to_categorical(y_train, 2))
y_val = np.array(np_utils.to_categorical(y_val, 2))

model = create_model(input_shape)

model.compile(optimizer="Adam", 
      loss='categorical_crossentropy', 
      metrics=['accuracy'])

model.fit(x=x_train,
          y=y_train,
          epochs=25,
          batch_size=15,
          validation_data=(x_val, y_val))

save_model(model, 
   'asthma_VGG19_0', 
   overwrite=True, 
   include_optimizer=True, 
   save_format=None, 
   signatures=None, options=None, save_traces=True)
        

Epoch 1/25
10/10 [==============================] - 41s 4s/step - loss: 0.7076 - accuracy: 0.4388 - val_loss: 0.7049 - val_accuracy: 0.4857
Epoch 2/25
10/10 [==============================] - 39s 4s/step - loss: 0.7158 - accuracy: 0.5396 - val_loss: 0.6811 - val_accuracy: 0.5143
Epoch 3/25
10/10 [==============================] - 39s 4s/step - loss: 0.7164 - accuracy: 0.5036 - val_loss: 0.7063 - val_accuracy: 0.4571
Epoch 4/25
10/10 [==============================] - 39s 4s/step - loss: 0.7725 - accuracy: 0.4532 - val_loss: 0.7289 - val_accuracy: 0.4000
Epoch 5/25
10/10 [==============================] - 41s 4s/step - loss: 0.7436 - accuracy: 0.4892 - val_loss: 0.6911 - val_accuracy: 0.5714
Epoch 6/25
10/10 [==============================] - 53s 5s/step - loss: 0.7126 - accuracy: 0.5324 - val_loss: 0.7167 - val_accuracy: 0.4286
Epoch 7/25
 8/10 [=======================>......] - ETA: 8s - loss: 0.7161 - accuracy: 0.4500 

KeyboardInterrupt: 